<h2>Import Packages and Data</h2>

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
data = pd.read_csv("/Users/quintonaguilera/Desktop/Quinton | Aldo | Luigi/landslide_data.csv") 

<h2>Define Functions</h2>

Not all functions will be explained in great detail, but the build_multi_output_model function will, because of its importance throughout the notebook:

The function builds a neural network comprised of essentially 5 parts:

1) The input layer

2) The first collection of hidden layers

3) The first output

4) The second collection of hidden layers

5) The second output


Each non-output layer has a dropout layer following it (dropout rate = 0.2), and all layers have an orthagonal kernel initializer, with the exceptino of the input layer. L1 and L2 regularisation is used for all hidden layers. 

The function requires being passed the number of layers in the first collection of hidden layers (stage_1_hidden_layers), and the number of neurons in each layer (stage_1_neurons), as well as the same for the second stage of hidden layers. The activation functions for the first stage are rely, and 'leaky relu' for the second stage (this was done to combat the somehat common "dying relu" problem). The activation functions for the outputs are sigmoid and softmax, corresponding to the output formats required. 

In [ ]:
def build_muti_output_model(stage_1_hidden_layers, stage_2_hidden_layers, 
                            stage_1_neurons, stage_2_neurons):
    input_size=19
    stage1_counter = 0
    stage2_counter = 0
    
    input_layer = keras.layers.Input(input_size, 
                                     name="input")

    while stage1_counter < (stage_1_hidden_layers):
        if stage1_counter == 0:
            x = keras.layers.Dense(stage_1_neurons, activation="relu", 
                                   kernel_initializer=tf.keras.initializers.Orthogonal(),
                                   kernel_regularizer=regularizers.l1_l2(l1=1e-5, l2=1e-4)
                                  )(input_layer)
            x = keras.layers.Dropout(0.2)(x)
            stage1_counter += 1
        else:
            x = keras.layers.Dense(stage_1_neurons, activation="relu",
                                  kernel_initializer=tf.keras.initializers.Orthogonal(),
                                  kernel_regularizer=regularizers.l1_l2(l1=1e-5, l2=1e-4)
                                  )(x)
            x = keras.layers.Dropout(0.2)(x)
            stage1_counter += 1
        
    count_output = keras.layers.Dense(1, activation="sigmoid",
                                      kernel_initializer=tf.keras.initializers.Orthogonal(),
                                      name="count_output")(x)
    
    while stage2_counter < (stage_2_hidden_layers):
        if stage1_counter == 0:
            x = keras.layers.Dense(stage_2_neurons, activation=tf.keras.layers.LeakyReLU(alpha=0.1), 
                                   kernel_initializer=tf.keras.initializers.Orthogonal(),
                                   kernel_regularizer=regularizers.l1_l2(l1=1e-5, l2=1e-4))(count_output)
            x = keras.layers.Dropout(0.2)(x)
            stage2_counter += 1
        else:
            x = keras.layers.Dense(stage_2_neurons, activation=tf.keras.layers.LeakyReLU(alpha=0.1),
                                  kernel_initializer=tf.keras.initializers.Orthogonal(),
                                  kernel_regularizer=regularizers.l1_l2(l1=1e-5, l2=1e-4))(x)
            x = keras.layers.Dropout(0.2)(x)
            stage2_counter += 1
    
    class_output = keras.layers.Dense(4, activation="softmax",
                                      kernel_initializer=tf.keras.initializers.Orthogonal(),
                                      name="class_output")(x)


    model = keras.Model(inputs=input_layer, outputs=[count_output, class_output]) 

    return model

In [2]:
# Monte carlo approach to finding optimal hyper parameters

def monte_carlo_tuning(x_train, y_train_1, y_train_2, 
                       first_stage_hl, second_stage_hl,
                       first_stage_n, second_stage_n,
                       runs=10, n_samples=1):
    min_loss = 0.0
    min_count_loss = 0.0
    min_area_loss = 0.0
    max_conf = []

    es = tf.keras.callbacks.EarlyStopping(monitor='val_loss', 
                                          mode='min', verbose=0, patience=0, restore_best_weights=True)

    config_cache_loss = {}
    config_cache_count = {}
    config_cache_area = {}
    
    for _ in tqdm(range(runs)):
        
        configuration = generate_configuration(first_stage_hl, second_stage_hl,
                                      first_stage_n, second_stage_n)
        
        if all(configuration) not in config_cache_loss:
            loss = 0.0
            count_loss = 0.0
            area_loss = 0.0
            
            for _ in range(n_samples):
                model = build_muti_output_model(configuration[0], configuration[1],
                                               configuration[2], configuration[3])

                model.compile(loss=["binary_crossentropy", "mse"],
                              optimizer="adam")


                history = model.fit(x_train, [y_train_1, y_train_2], 
                    epochs=5,
                    validation_split=0.2,
                    callbacks=[keras.callbacks.EarlyStopping(patience=15)],
                    verbose=0)
                
                loss       += history.history['val_loss'][-1]
                count_loss += history.history['val_count_output_loss'][-1]
                area_loss  += history.history['val_area_output_loss'][-1]
            loss        = loss/n_samples
            count_loss  = count_loss/n_samples
            area_loss   = area_loss/n_samples

            config_cache_loss[str(configuration)]  = loss
            config_cache_count[str(configuration)] = count_loss
            config_cache_area[str(configuration)]  = area_loss

    return config_cache_loss, config_cache_count, config_cache_area

In [ ]:
# given a datalist and a number of breakpoints, this function finds the break point values such that the
# variance in each category is minimised

def getJenksBreaks(dataList, numClass):
    dataList = list(dataList)
    dataList.sort()
    mat1 = []
    for i in range(0,len(dataList)+1):
        temp = []
        for j in range(0,numClass+1):
            temp.append(0)
        mat1.append(temp)
    mat2 = []
    for i in range(0,len(dataList)+1):
        temp = []
        for j in range(0,numClass+1):
            temp.append(0)
        mat2.append(temp)
    for i in range(1,numClass+1):
        mat1[1][i] = 1
        mat2[1][i] = 0
        for j in range(2,len(dataList)+1):
            mat2[j][i] = float('inf')
    v = 0.0
    for l in range(2,len(dataList)+1):
        s1 = 0.0
        s2 = 0.0
        w = 0.0
        for m in range(1,l+1):
            i3 = l - m + 1
            val = float(dataList[i3-1])
            s2 += val * val
            s1 += val
            w += 1
            v = s2 - (s1 * s1) / w
            i4 = i3 - 1
            if i4 != 0:
                for j in range(2,numClass+1):
                    if mat2[l][j] >= (v + mat2[i4][j - 1]):
                        mat1[l][j] = i3
                        mat2[l][j] = v + mat2[i4][j - 1]
        mat1[l][1] = 1
        mat2[l][1] = v
    k = len(dataList)

    breaks = []
    for i in range(0,numClass+1):
        breaks.append(min(dataList))
    countNum = numClass

    while countNum >= 2:#print "rank = " + str(mat1[k][countNum])
        id = int((mat1[k][countNum]) - 2)
        value = dataList[id]
        breaks.append(value)
        k = int((mat1[k][countNum] - 1))
        countNum -= 1

    breaks.append(max(dataList))
    breaks = list(set(breaks))
    breaks.sort()
    return(breaks)

    # The first output number is the smallest value in the input
    # Following output numbers are inclusive upper bounds

In [ ]:
# These functions are used to determine the goodness of variance fit, determining how well the function above
# performed (in other words, it measures how well using a specific number of breakpoints applies to the data)

def goodness_of_variance_fit(array, nclasses):
    
        classes = getJenksBreaks(array, nclasses)
        classified = np.array([classify(i, classes) for i in array])
        maxz = max(classified)
        zone_indices = [[idx for idx, val in enumerate(classified) if zone + 1 == val] for zone in range(maxz)]
        sdam = np.sum((array - array.mean()) ** 2)
        array_sort = [np.array([array[index] for index in zone]) for zone in zone_indices]
        sdcm = sum([np.sum((classified - classified.mean()) ** 2) for classified in array_sort])
        gvf = (sdam - sdcm) / sdam
    
        return gvf, classes
    
def classify(value, breaks):
    for i in range(1, len(breaks)):
        if value < breaks[i]:
            return i
    return len(breaks) - 1

In [ ]:
# This function iterates over various numbers of breakpoints and uses the function above to find the best 
# number of break points

def optimal_classes(data, gvf_threshold, min_classes, max_classes):

    gvf= 0.0
    nclasses = min_classes

    while gvf < gvf_threshold:
            gvf, classes = goodness_of_variance_fit(data, nclasses)
            if gvf < gvf_threshold:
                gvf, classes = goodness_of_variance_fit(data, nclasses)
                print('Classes Trialed: ', nclasses)
                print('gvf value :', gvf)
                print('Outcome: gvf value insufficient, further trials will be attempted')
                print()
                nclasses += 1
            else:
                print('Classes Trialed: ', nclasses)
                print('gvf value: ', gvf)
                print('Breaks: ', classes)
                print('Outcome: Ladies and gentlemen, we got him.')
                print()

<h2>Data Preprocessing</h2>

The first step is to determine the natural breaks of the Area_Slide variable. The first class will be records with an area of 0 (no landslide). Of the remaining records, natural breaks in the data will be determined. 

In [ ]:
# Create a df of non-zero Area_Slide
filtered_data = data[data['Area_Slide'] > 0]
non_zero_slide_area = filtered_data['Area_Slide']
non_zero_slide_area = np.array(non_zero_slide_area)

In [ ]:
# Find the optimal breaks in non-zero Area_Slide
optimal_classes(non_zero_slide_area, 0.85, 2, 8)

Use these breaks to create a new column in the main dataframe applying a class to each record.

In [ ]:
data.loc[(data["Area_Slide"] > 0) & (data["Area_Slide"] < 0.054691), "Slide_Class"] = "Class 0" 
data.loc[(data["Area_Slide"] >= 0.054691) & (data["Area_Slide"] <= 2981.98645), "Slide_Class"] = "Class 1" 
data.loc[(data["Area_Slide"] > 2981.98645) & (data["Area_Slide"] <= 10821.34832), "Slide_Class"] = "Class 2" 
data.loc[(data["Area_Slide"] > 10821.34832) & (data["Area_Slide"] <= 29509.81466), "Slide_Class"] = "Class 3" 
data.loc[(data["Area_Slide"] > 29509.81466) & (data["Area_Slide"] <= 82850.15392), "Slide_Class"] = "Class 4"
data.loc[(data["Area_Slide"] > 82850.15392) & (data["Area_Slide"] <= 182966.8526), "Slide_Class"] = "Class 5"
data["Slide_Class"].value_counts()

In [ ]:
# See the value_counts of each class
data["Slide_Class"].value_counts()

There are two major observations here:
1) There is a large class imbalance, this will negatively affect training data. Oversampling will take place later to fix this problem.


2) Oversampling requires a number of similar neighbors, so classes with inadequate value counts will be merged to facilitate oversampling.

In [ ]:
data.loc[(data["Area_Slide"] > 0) & (data["Area_Slide"] < 0.054691), "Slide_Class"] = "Class 0" 
data.loc[(data["Area_Slide"] >= 0.054691) & (data["Area_Slide"] <= 2981.98645), "Slide_Class"] = "Class 1" 
data.loc[(data["Area_Slide"] > 2981.98645) & (data["Area_Slide"] <= 10821.34832), "Slide_Class"] = "Class 1" 
data.loc[(data["Area_Slide"] > 10821.34832) & (data["Area_Slide"] <= 29509.81466), "Slide_Class"] = "Class 2" 
data.loc[(data["Area_Slide"] > 29509.81466) & (data["Area_Slide"] <= 82850.15392), "Slide_Class"] = "Class 3"
data.loc[(data["Area_Slide"] > 82850.15392) & (data["Area_Slide"] <= 182966.8526), "Slide_Class"] = "Class 3"
data["Slide_Class"].value_counts()

In [ ]:
# Define dependant and explanatory variables

data_y = data['Slide_Class']
data_x = data.drop(['SU_ID', 'Count', 'Area_Slide', 'Slide_Class'], axis=1)
data_x = data_x.astype(float)

In [ ]:
# divide into training and test datasets
from sklearn.model_selection import train_test_split
x_train, x_test, y_train_area, y_test_area = train_test_split(data_x, data_y, test_size=0.2, random_state=0)

In [ ]:
# Standardize the explanatory variables

from sklearn import preprocessing

scaler = preprocessing.StandardScaler()

train_x_values = x_train.values
x_scaled1 = scaler.fit_transform(train_x_values)
x_train = pd.DataFrame(x_scaled1)

test_x_values = x_test.values
x_scaled2 = scaler.fit_transform(test_x_values)
x_test = pd.DataFrame(x_scaled2)

In [ ]:
# Use SMOTE package to oversample training data
from imblearn.over_sampling import SMOTE
oversample = SMOTE()
x_train, y_train_area = oversample.fit_resample(x_train, y_train_area)

In [ ]:
# Use the Slide_Class field to create a binary variable that 
# indicates whether or not there is a landslide in any capacity

y_train_count = []
y_test_count = []

for i in y_train_area:
    if i == "Class 0":
        y_train_count.append(0)
    else:
        y_train_count.append(1)
        
for i in y_test_area:
    if i == "Class 0":
        y_test_count.append(0)
    else:
        y_test_count.append(1)
        
y_train_count = np.array(y_train_count)
y_test_count = np.array(y_test_count)

The last step of preprocessing is to encode the area classes for input into the neural network

In [ ]:
from sklearn.preprocessing import LabelEncoder
from keras.utils import np_utils
# encode class values as integers
encoder = LabelEncoder()

#training
encoder.fit(y_train_area)
encoded_Y = encoder.transform(y_train_area)
# convert integers to dummy variables (i.e. one hot encoded)
y_train = np_utils.to_categorical(encoded_Y)

#testing
encoder.fit(y_test_area)
encoded_Y_test = encoder.transform(y_test_area)
y_test = np_utils.to_categorical(encoded_Y_test)

<h2>Modelling Process</h2>

In [ ]:
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Input
from keras.utils.vis_utils import plot_model
from keras.initializers import RandomNormal
import tensorflow as tf
import tensorflow.keras.optimizers
from tensorflow.keras import initializers
from tensorflow.keras import regularizers
from keras.layers import LeakyReLU
from keras.regularizers import l2
from keras.layers import Dropout

In [ ]:
### USE MONTE CARLO TUNING
### This process takes hours, so if you are interested in running this code, please 
### use the placeholder values listed below:

### placeholder = (6,6,28,4)

In [ ]:
model = build_muti_output_model(6, 6, 28, 4)
opt = tf.keras.optimizers.Adam(learning_rate=3e-2)
model.compile(loss=["binary_crossentropy", "categorical_crossentropy"], 
              optimizer='adam')
model.summary()

In [ ]:
history = model.fit(x_train, [y_train_count, y_train], 
                    epochs=300,
                    validation_split=0.2,
                    callbacks=[keras.callbacks.EarlyStopping(patience=15)])

<h2>Training Evaluation</h2>

<h3>Training: Count Output</h3>

In [ ]:
# plot loss during training
import matplotlib.pyplot as plt
plt.subplot()
plt.title('Count loss ')
plt.plot(history.history['count_output_loss'], label='loss')
plt.plot(history.history['val_count_output_loss'], label='val')
plt.legend()

In [ ]:
# plot loss during training
import matplotlib.pyplot as plt
plt.subplot()
plt.title('Area loss ')
plt.plot(history.history['class_output_loss'], label='loss')
plt.plot(history.history['val_class_output_loss'], label='val')
plt.legend()

<h3>Training: Area Output</h3>

<h2>Evaluation Using Test Data</h2>

In [ ]:
y_pred=model.predict(x_test)
count_predictions = y_pred[0]
area_predictions = y_pred[1]

Typically, when evaluating classifiers on imbalanced data, ROC AUC is not the preffered evaluation metric. Instead, a precision and recal curve would be used. In this case, this model was designed to outperform a statistical approach to the same problem (not shown in this notebook). The paper in question used ROC AUC to evaluate its perfromance (perhaps not the ideal measure) but in light of this, ROC AUC is also used here to enable direct comparison. 

<h3>Test: Count Output</h3>

In [1]:
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
# calculate scores
count_auc = roc_auc_score(y_test_count, count_predictions)
# summarize scores
print('NN: ROC AUC=%.6f' % (count_auc))

NameError: name 'y_test_count' is not defined

In [ ]:
ns_probs = [0 for _ in range(len(y_test_count))]
ns_fpr, ns_tpr, _ = roc_curve(y_test_count, ns_probs)
lr_fpr, lr_tpr, _ = roc_curve(y_test_count, count_predictions)
plt.plot(ns_fpr, ns_tpr, linestyle='--', label='No Skill')
plt.plot(lr_fpr, lr_tpr, marker='.', label='Logistic')
# axis labels
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
# show the legend
plt.legend()
# show the plot
plt.show()

In [ ]:
from sklearn.metrics import precision_recall_curve
precision, recall, thresholds = precision_recall_curve(y_test_count, count_predictions)

fig, ax = plt.subplots(figsize=(6,6))
ax.plot(recall, precision, label='Logistic Regression')
baseline = len(y_test_count[y_test_count==1]) / len(y_test_count)
ax.plot([0, 1], [baseline, baseline], linestyle='--', label='Baseline')
ax.set_xlabel('Recall')
ax.set_ylabel('Precision')
ax.legend(loc='center left');
print("Mean Precision: ",np.mean(precision))
print("Mean Recall:    ", np.mean(recall))

<h3>Test: Area Output</h3>

In [ ]:
macro_roc_auc_ovo = roc_auc_score(y_test_area, area_predictions, multi_class="ovo",
                                  average="macro")
weighted_roc_auc_ovo = roc_auc_score(y_test_area, area_predictions, multi_class="ovo",
                                     average="weighted")
macro_roc_auc_ovr = roc_auc_score(y_test_area, area_predictions, multi_class="ovr",
                                  average="macro")
weighted_roc_auc_ovr = roc_auc_score(y_test_area, area_predictions, multi_class="ovr",
                                     average="weighted")
print("One-vs-One ROC AUC scores:\n{:.6f} (macro),\n{:.6f} "
      "(weighted by prevalence)"
      .format(macro_roc_auc_ovo, weighted_roc_auc_ovo))
print("One-vs-Rest ROC AUC scores:\n{:.6f} (macro),\n{:.6f} "
      "(weighted by prevalence)"
      .format(macro_roc_auc_ovr, weighted_roc_auc_ovr))

In [ ]:
from sklearn.metrics import roc_curve, auc
def plot_multiclass_roc(y_test, y_pred, n_classes, figsize=(17, 6)):

    # structures
    fpr = dict()
    tpr = dict()
    roc_auc = dict()

    # calculate dummies once
    y_test_dummies = pd.get_dummies(y_test, drop_first=False).values
    for i in range(n_classes):
        fpr[i], tpr[i], _ = roc_curve(y_test_dummies[:, i], y_pred[:, i])
        roc_auc[i] = auc(fpr[i], tpr[i])

    # roc for each class
    fig, ax = plt.subplots(figsize=figsize)
    ax.plot([0, 1], [0, 1], 'k--')
    ax.set_xlim([0.0, 1.0])
    ax.set_ylim([0.0, 1.05])
    ax.set_xlabel('False Positive Rate')
    ax.set_ylabel('True Positive Rate')
    ax.set_title('ROC Curves for Each Class')
    for i in range(n_classes):
        ax.plot(fpr[i], tpr[i], label='ROC curve (area = %0.6f) for label %i' % (roc_auc[i], i))
    ax.legend(loc="best")
    ax.grid(alpha=.4)
    plt.show()

In [ ]:
plot_multiclass_roc(y_test_area, area_predictions, n_classes=4, figsize=(12, 9))
# (4, 6, 16, 6)
# 0.87, 0.81, 0.88, 0.96

Lambda Experiment

In [ ]:
model = build_muti_output_model(6, 6, 28, 4)
opt = tf.keras.optimizers.Adam(learning_rate=3e-2)
model.compile(loss=["binary_crossentropy", "categorical_crossentropy"], 
              optimizer='adam',
              loss_weights={"count_output": 0.6, "class_output": 0.4})
history = model.fit(x_train, [y_train_count, y_train], 
                    epochs=300,
                    validation_split=0.2,
                    callbacks=[keras.callbacks.EarlyStopping(patience=15)])
y_pred=model.predict(x_test)
count_predictions = y_pred[0]
area_predictions = y_pred[1]
# 6,9,28,4 produced 0.901
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
# calculate scores
lr_auc = roc_auc_score(y_test_count, count_predictions)
# summarize scores
print('NN: ROC AUC=%.6f' % (lr_auc))

In [ ]:
def lamda_testing():
    weight_1 = 0
    weight_2 = 1-weight_1
    runs = 5
    while weight_1 <= 1:
        model = build_muti_output_model(6, 6, 28, 4)
        model.compile(loss=["binary_crossentropy", "categorical_crossentropy"], 
              optimizer='adam',
              loss_weights={"count_output": weight_1, "class_output": weight_2})
        for i in range(runs):
            history = model.fit(x_train, [y_train_count, y_train], 
                    epochs=300,
                    validation_split=0.2,
                    callbacks=[keras.callbacks.EarlyStopping(patience=15)],
                    verbose=0)
            y_pred=model.predict(x_test)
            count_predictions = y_pred[0]
            area_predictions = y_pred[1]
            lr_auc = roc_auc_score(y_test_count, count_predictions)
            # summarize scores
            print(weight_1, weight_2)
            print('NN: ROC AUC=%.6f' % (lr_auc))
        weight_1 += 0.2

In [ ]:
lamda_testing()